# Neural Network

In [1]:
!pip install pyyaml numpy scipy matplotlib ipython jupyter pandas sympy nose opencv-python opencv-contrib-python imutils

     |████████████████████████████████| 266kB 4.9MB/s eta 0:00:01
     |████████████████████████████████| 25.2MB 31.8MB/s eta 0:00:01
     |████████████████████████████████| 10.4MB 16.4MB/s eta 0:00:01    |████████████████████▎           | 6.6MB 16.4MB/s eta 0:00:01
     |████████████████████████████████| 5.3MB 22.2MB/s eta 0:00:01
     |████████████████████████████████| 163kB 39.5MB/s eta 0:00:01
     |████████████████████████████████| 28.7MB 28.2MB/s eta 0:00:01
     |████████████████████████████████| 34.7MB 1.2MB/s  eta 0:00:01
     |████████████████████████████████| 512kB 29.4MB/s eta 0:00:01
     |████████████████████████████████| 522kB 29.5MB/s eta 0:00:01


  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=45393 sha256=aaf6725e2e693cdfa305c5742c54b522d3bb503be8370354071dda92d0568700
  Stored in directory: /root/.cache/pip/wheels/d9/45/dd/65f0b38450c47cf7e5312883deb97d065e030c5cca0a365030
  Created wheel for imutils: filename=imutils-0.5.3-cp36-none-any.whl size=26734 sha256=d8a837b0849f7e111ee2aeb7c46ed429e7ebf5f9b57253211963b4bf889eb9fc
  Stored in directory: /root/.cache/pip/wheels/16/84/1f/bf88641293cda2c8be81a5c4b8ca973dd9125a6dc3767417fd
  Created wheel for mpmath: filename=mpmath-1.1.0-cp36-none-any.whl size=532006 sha256=4bf41778a0dd8d674b142ed6212ec02a1d5754a31cc9882dd0123eb6ce2b431a
  Stored in directory: /root/.cache/pip/wheels/63/9d/8e/37c3f6506ed3f152733a699e92d8e0c9f5e5f01dea262f80ad
Successfully built pyyaml imutils mpmath
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd

import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

import yaml

import matplotlib.pyplot as plt

In [3]:
hparams = yaml.safe_load(open('../src/models/hparams.yaml'))

hparams

{'learning_rate': 0.01, 'batch_size': 16, 'num_hidden_layers': 4}

In [4]:
wiki_df = pd.read_csv('../data/processed/wiki_df.csv', sep=';')

wiki_df.head()

,full_path,gender,age,img_array
0,17/10000217_1981-05-05_2009.jpg,1.0,28,[255 255 255 ... 144 78 27]
1,12/100012_1948-07-03_2008.jpg,1.0,60,[92 98 93 ... 35 31 30]
2,16/10002116_1971-05-31_2012.jpg,0.0,41,[ 10 30 61 ... 231 237 255]
3,02/10002702_1960-11-09_2012.jpg,0.0,52,[178 122 97 ... 168 112 83]
4,41/10003541_1937-09-27_1971.jpg,1.0,34,[194 189 190 ... 101 103 104]


In [5]:
import cv2

wiki_df['img_array'] = wiki_df['full_path'].apply(lambda x: cv2.resize(cv2.imread('../data/raw/wiki_crop/' + x), (224, 224), interpolation=cv2.INTER_LINEAR).reshape(1, -1)[0])

## Experiment 1

* Ako prvé sa pokúsime vytvoriť NN podobnú VGG. Podľa [WEEK_7 lab](https://github.com/matus-pikuliak/neural_networks_at_fiit/blob/92b24eef8e6444c43a22e8fa51a349b3b1043a7c/week_7/week_7.ipynb), alebo iného tutoriálu
* Natrénujeme ju na už predspracovanom datasete
* Jej výsledok pou%zijeme ako štartovaciu čiaru
* Túto sieť budeme rozširovať o ďalšie vrstvy a parametre
* V projekte ponecháme sieť s najlepším skóre
* Dole pripájam referenčnú ukážku siete. [Zdroj](https://www.pyimagesearch.com/2019/10/28/3-ways-to-create-a-keras-model-with-tensorflow-2-0-sequential-functional-and-model-subclassing/?__s)

In [5]:
class MiniVGGNetModel(keras.Model):
    def __init__(self, classes, chanDim=-1):
        # call the parent constructor
        super(MiniVGGNetModel, self).__init__()

        # initialize the layers in the first (CONV => RELU) * 2 => POOL
        # layer set
        self.conv1A = Conv2D(32, (3, 3), padding="same")
        self.act1A = Activation("relu")
        self.bn1A = BatchNormalization(axis=chanDim)
        self.conv1B = Conv2D(32, (3, 3), padding="same")
        self.act1B = Activation("relu")
        self.bn1B = BatchNormalization(axis=chanDim)
        self.pool1 = MaxPooling2D(pool_size=(2, 2))

        # initialize the layers in the second (CONV => RELU) * 2 => POOL
        # layer set
        self.conv2A = Conv2D(32, (3, 3), padding="same")
        self.act2A = Activation("relu")
        self.bn2A = BatchNormalization(axis=chanDim)
        self.conv2B = Conv2D(32, (3, 3), padding="same")
        self.act2B = Activation("relu")
        self.bn2B = BatchNormalization(axis=chanDim)
        self.pool2 = MaxPooling2D(pool_size=(2, 2))

        # initialize the layers in our fully-connected layer set
        self.flatten = Flatten()
        self.dense3 = Dense(512)
        self.act3 = Activation("relu")
        self.bn3 = BatchNormalization()
        self.do3 = Dropout(0.5)

        # initialize the layers in the softmax classifier layer set
        self.dense4 = Dense(classes)
        self.softmax = Activation("softmax")

    def call(self, inputs):
        # build the first (CONV => RELU) * 2 => POOL layer set
        x = self.conv1A(inputs)
        x = self.act1A(x)
        x = self.bn1A(x)
        x = self.conv1B(x)
        x = self.act1B(x)
        x = self.bn1B(x)
        x = self.pool1(x)

        # build the second (CONV => RELU) * 2 => POOL layer set
        x = self.conv2A(x)
        x = self.act2A(x)
        x = self.bn2A(x)
        x = self.conv2B(x)
        x = self.act2B(x)
        x = self.bn2B(x)
        x = self.pool2(x)

        # build our FC layer set
        x = self.flatten(x)
        x = self.dense3(x)
        x = self.act3(x)
        x = self.bn3(x)
        x = self.do3(x)

        # build the softmax classifier
        x = self.dense4(x)
        x = self.softmax(x)

        # return the constructed model
        return x

# Experiment 2
Cely VGG model.

Zdroj: https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/

In [6]:
model = keras.models.Sequential()
model.add(keras.layers.ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(keras.layers.Convolution2D(64, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(128, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(128, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(256, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(256, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(256, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(512, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(512, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(512, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(512, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(512, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(512, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
model.add(keras.layers.Convolution2D(4096, (7, 7), activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Convolution2D(4096, (1, 1), activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Convolution2D(2622, (1, 1)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Activation('softmax'))

In [7]:
model.load_weights('../data/vgg_face_weights.h5')

In [8]:
vgg_face_descriptor = keras.Model(inputs=model.layers[0].input, outputs=model.layers[-1].output)

Convert image as array using opencv

In [9]:
classes = 101 #0 to 100
target = wiki_df['age'].values
target_classes = keras.utils.to_categorical(target, classes)

features = []
 
for i in range(0, wiki_df.shape[0]):
    features.append(wiki_df['img_array'].values[i])
 
features = np.array(features)
features = features.reshape(features.shape[0], 224, 224, 3)

In [12]:
!pip install scikit-learn

     |████████████████████████████████| 6.7MB 4.7MB/s eta 0:00:01
     |████████████████████████████████| 296kB 28.7MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [13]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(features, target_classes, test_size=0.30)

In [14]:
for layer in model.layers[:-7]:
    layer.trainable = False

In [15]:
base_model_output = keras.models.Sequential()
base_model_output = keras.layers.Convolution2D(101, (1, 1), name='predictions')(model.layers[-4].output)
base_model_output = keras.layers.Flatten()(base_model_output)
base_model_output = keras.layers.Activation('softmax')(base_model_output)

In [16]:
age_model = keras.Model(inputs=model.input, outputs=base_model_output)

In [17]:
age_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [ ]:
scores = []
epochs = 250; batch_size = 50
 
for i in range(epochs):
    print("epoch ",i)
 
    ix_train = np.random.choice(train_x.shape[0], size=batch_size)
 
    score = age_model.fit(train_x[ix_train], train_y[ix_train], epochs=1, validation_data=(test_x, test_y))
 
    scores.append(score)

epoch  0
